In [19]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore')

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import *

import shap
from pdpbox.pdp import pdp_isolate, pdp_plot

ImportError: cannot import name 'pdp_isolate' from 'pdpbox.pdp' (/Users/bumsoojoe/anaconda3/envs/py/lib/python3.8/site-packages/pdpbox/pdp.py)

In [2]:
df = pd.read_csv('/Users/bumsoojoe/Desktop/kaggel project/Data/train_data.csv')

In [3]:
df

,sequence_id,sequence,experiment_type,dataset_name,reads,signal_to_noise,SN_filter,reactivity_0001,reactivity_0002,reactivity_0003,...,reactivity_error_0197,reactivity_error_0198,reactivity_error_0199,reactivity_error_0200,reactivity_error_0201,reactivity_error_0202,reactivity_error_0203,reactivity_error_0204,reactivity_error_0205,reactivity_error_0206
0,8cdfeef009ea,GGGAACGACUCGAGUAGAGUCGAAAAACGUUGAUAUGGAUUUACUC...,2A3_MaP,15k_2A3,2343,0.944,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,51e61fbde94d,GGGAACGACUCGAGUAGAGUCGAAAAACAUUGAUAUGGAUUUACUC...,2A3_MaP,15k_2A3,5326,1.933,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25ce8d5109cd,GGGAACGACUCGAGUAGAGUCGAAAAACCUUGAUAUGGAUUUACUC...,2A3_MaP,15k_2A3,4647,2.347,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,07dcfb6d1965,GGGAACGACUCGAGUAGAGUCGAAAAACUUUGAUAUGGAUUUACUC...,2A3_MaP,15k_2A3,102843,11.824,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,e561cc042a4c,GGGAACGACUCGAGUAGAGUCGAAAAACGAUGAUAUGGAUUUACUC...,2A3_MaP,15k_2A3,7665,3.519,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1643675,7951fb2f47f1,GGGAACGACUCGAGUAGAGUCGAAAAGGAGCGUCGUGUCUCUUGUA...,DMS_MaP,SL5_M2seq_DMS,37530,7.248,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1643676,e0dc5823e5e1,GGGAACGACUCGAGUAGAGUCGAAAAGGAGCGUCGUGUCUCUUGUA...,DMS_MaP,SL5_M2seq_DMS,337248,17.902,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1643677,0d6036529b42,GGGAACGACUCGAGUAGAGUCGAAAAGGAGCGUCGUGUCUCUUGUA...,DMS_MaP,SL5_M2seq_DMS,44053,6.700,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1643678,46d1f07d723b,GGGAACGACUCGAGUAGAGUCGAAAAGGAGCGUCGUGUCUCUUGUA...,DMS_MaP,SL5_M2seq_DMS,108600,11.716,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# 하나의 서열이 같은 것이 존재하는가?
test1 = df['sequence'][0]
test2 = df['sequence'][0]
s = 0
n = 0
for i in range(0,len(test1)):
    if test1[i] == test2[i]:
        s += 1
    else:
        n += 1
print('같은 염기서열 수 :',s)
print('다른 염시서열 수 :',n)

같은 염기서열 수 : 170
다른 염시서열 수 : 0


In [5]:
df['experiment_type'].value_counts()

experiment_type
2A3_MaP    821840
DMS_MaP    821840
Name: count, dtype: int64

In [6]:
# 같은 서열이 과연 몇개나 존재하는가?
test3 = list(df[df['experiment_type']=='2A3_MaP']['sequence'].value_counts().index)
print(len(list(df[df['experiment_type']=='2A3_MaP']['sequence'])) - len(test3))

15267


In [17]:
lenlist = []
for i in test3:
    lenlist.append(len(i))
max(lenlist)

206

In [7]:
len(test3)

806573

In [8]:
test4 = list(df[df['experiment_type']=='DMS_MaP']['sequence'].value_counts().index)
print(len(list(df[df['experiment_type']=='DMS_MaP']['sequence'])) - len(test3))

15267


In [18]:
lenlist2 = []
for i in test4:
    lenlist2.append(len(i))
max(lenlist2)

206

In [9]:
print(len(df[df['experiment_type']=='DMS_MaP']['dataset_name'].value_counts().index))

23


In [10]:
first_row = df[0:1]

# NaN이 아닌 컬럼만 반환
non_nan_columns = first_row.dropna(axis=1)

non_nan_columns

,sequence_id,sequence,experiment_type,dataset_name,reads,signal_to_noise,SN_filter,reactivity_0027,reactivity_0028,reactivity_0029,...,reactivity_error_0120,reactivity_error_0121,reactivity_error_0122,reactivity_error_0123,reactivity_error_0124,reactivity_error_0125,reactivity_error_0126,reactivity_error_0127,reactivity_error_0128,reactivity_error_0129
0,8cdfeef009ea,GGGAACGACUCGAGUAGAGUCGAAAAACGUUGAUAUGGAUUUACUC...,2A3_MaP,15k_2A3,2343,0.944,0,-0.039,0.455,-0.207,...,0.224,0.085,0.085,0.1,0.224,0.286,0.246,0.085,0.224,0.254


In [11]:
example_list = list(df[0:1].columns)[7:]

In [13]:
items_with_error = [item for item in example_list if "error" in item]
items_without_error = [item for item in example_list if "error" not in item]

In [14]:
len(items_with_error)

206

In [15]:
len(items_without_error)

206

In [12]:
df.isna().sum().sort_values()

sequence_id                    0
sequence                       0
experiment_type                0
dataset_name                   0
reads                          0
                          ...   
reactivity_error_0015    1643680
reactivity_error_0016    1643680
reactivity_error_0017    1643680
reactivity_error_0003    1643680
reactivity_error_0206    1643680
Length: 419, dtype: int64

In [23]:
df['reactivity_0001'].isna().sum()

1643680

In [29]:
df.shape[0]

1643680

In [30]:
ALLNaN = []
for i in example_list:
    if df[i].isna().sum() == df.shape[0]:
        ALLNaN.append(i)
print(len(ALLNaN))

134


In [31]:
test10 = pd.read_csv('/Users/bumsoojoe/Desktop/kaggel project/Data/test_sequences.csv')
test10

,id_min,id_max,sequence_id,sequence,future
0,0,176,eee73c1836bc,GGGAACGACUCGAGUAGAGUCGAAAAUUUCCUUCCAAAUCCUGAGG...,0
1,177,353,d2a929af7a97,GGGAACGACUCGAGUAGAGUCGAAAAUGUAAUCAGAUUGCUUCUCC...,0
2,354,530,d39a4425ff45,GGGAACGACUCGAGUAGAGUCGAAAAAACACAUGAAUUUGAGGGUU...,0
3,531,707,1fc41e92d553,GGGAACGACUCGAGUAGAGUCGAAAAUCAGAGCUGGCAAAUGGAUG...,0
4,708,884,1d0826fb892f,GGGAACGACUCGAGUAGAGUCGAAAAUUUGGUAUUUGAUGCAUUAA...,0
...,...,...,...,...,...
1343818,269794386,269794842,3cf515909422,GGGAACGACUCGAGUAGAGUCGAAAAGGAGGGAAAAGUUAUCAGGC...,1
1343819,269794843,269795299,3606852d6c2b,GGGAACGACUCGAGUAGAGUCGAAAAGGAGGGAAAAGUUAUCAGGC...,1
1343820,269795300,269795756,49b9448fd455,GGGAACGACUCGAGUAGAGUCGAAAAGGAGGGAAAAGUUAUCAGGC...,1
1343821,269795757,269796213,eb9bd6372550,GGGAACGACUCGAGUAGAGUCGAAAAGGAGGGAAAAGUUAUCAGGC...,1
